In [10]:
from pydub import AudioSegment, effects
from pydub.playback import play
import os
import shutil
import re

from scipy.io import wavfile
from pesq import pesq
import numpy as np

# Extracting unique characters from a dataset

In [1]:
#file_path = "/mnt/resource/Etc/data/adrita18/metadata.csv"
#file_path = "/mnt/resource/Etc/data/alamgir12/metadata.csv"
file_path = "/media/arif/Resource/Etc/data/alamgir12/metadata.csv"


text = ""
with open(file_path) as f:
    for i, line in enumerate(f):
        _, t, nt = line.split("|")
        text += nt.strip()

        if i < 5:
            print(nt)

characters = set(text)

print(f'Total unique characters: {len(characters)}')
print(f'Characters: {"".join(sorted(characters))}')

 স্কুলটির দিকে দৃষ্টি দিন।

 পিরোজপুর জেলার কাউখালী বন্দরে একটি মাত্র বালিকা বিদ্যালয় আছে, যা এসবি সরকারি উচ্চবালিকা বিদ্যালয় নামে পরিচিত।

 অনেক ঐতিহ্য বহন করছে এ বিদ্যালয়টি।

 কিন্তু এখানকার ছাত্রী ও শিক্ষক-শিক্ষিকাদের সম্মুখীন হতে হচ্ছে বহুবিধ সমস্যার!

 পাশ্বর্বর্তী বিভিন্ন গ্রাম ও দূরদূরান্ত থেকে ভালো শিক্ষার আশায় এখানে পড়তে আসছে শিক্ষার্থীরা।

Total unique characters: 97
Characters:  !"'(),-./:;?[] ।ঁংঃঅআইঈউঊঋএঐওঔকখগঘঙচছজঝঞটঠডঢণতথদধনপফবভমযরলশষসহ়ািীুূৃেৈোৌ্ৎৗড়ঢ়য়০১২৩৪৬৭৮৯‌‍–—―’”−


In [2]:
alamgir_text = text
len(alamgir_text)

660437

In [7]:
adrita_text = text
len(adrita_text)

793163

# Creating `characters` for COQUI-TTS

In [9]:
text = alamgir_text + adrita_text
chars = set(text)
print(f"Characters in corpus: {chars}")

Characters in corpus: {'ক', 'ড়', '৪', 'ফ', '৫', ' ', ')', 't', 'ৎ', '%', 'ঞ', 'ছ', 'য়', 'ভ', 'শ', 'a', 'ূ', '১', 'H', 'R', 'হ', 'ঁ', 'l', ',', 'F', 'ো', '.', 'া', 'র', '"', 'ষ', 'ৗ', 'প', 'h', 'ঠ', '0', '০', 'Y', 'ণ', '(', 'b', 'L', 'ঙ', 'ঐ', 'থ', '৯', 'x', '*', 'ঔ', '৬', 'য', 'k', 'ধ', 'C', '\u200d', 'খ', 'ই', '3', 'ঌ', 'B', '?', 'জ', '[', 'i', 'গ', 'ী', 'ৈ', 'উ', 'c', 'ঝ', 'I', 'ে', '8', 'e', 'ঢ়', 'M', 'ত', 'G', '২', '#', 'ু', '5', '৭', 'S', 'ৃ', '\xa0', '-', 'K', '৩', 'f', 'o', 'q', 'ও', '―', 'y', 'ঃ', 'ং', 'd', 'm', '/', 'T', '়', 'ব', 'স', '6', 'চ', '্', 'A', 'আ', '–', 'n', ';', 'u', '\u200c', '−', 'ন', 'w', 'E', 's', '!', 'r', '°', '—', 'ঈ', 'ি', 'J', '@', '৮', '’', 'ৌ', 'ড', 'অ', 'g', 'ঘ', '”', 'ম', 'D', ']', 'ঋ', ':', 'এ', 'ট', 'N', 'ঢ', 'ঊ', 'P', 'p', '1', 'ল', '।', "'", 'দ'}


In [48]:
chars = "@ABCDEFGHIJKLMNPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz"
puncs = ";:,.!?—…\"«»“” "

In [9]:
for c in chars:
    #if ord(c) < 128: continue # ASCII
        
    print(f"{c} -> \\u{ord(c):04x}")

0 -> \u0030
1 -> \u0031
2 -> \u0032
3 -> \u0033
4 -> \u0034
5 -> \u0035
6 -> \u0036
7 -> \u0037
8 -> \u0038
9 -> \u0039
@ -> \u0040
A -> \u0041
B -> \u0042
C -> \u0043
D -> \u0044
E -> \u0045
F -> \u0046
G -> \u0047
H -> \u0048
I -> \u0049
J -> \u004a
K -> \u004b
L -> \u004c
M -> \u004d
N -> \u004e
P -> \u0050
Q -> \u0051
R -> \u0052
S -> \u0053
T -> \u0054
U -> \u0055
V -> \u0056
W -> \u0057
X -> \u0058
Y -> \u0059
Z -> \u005a
a -> \u0061
b -> \u0062
c -> \u0063
d -> \u0064
e -> \u0065
f -> \u0066
g -> \u0067
h -> \u0068
i -> \u0069
j -> \u006a
k -> \u006b
l -> \u006c
m -> \u006d
n -> \u006e
o -> \u006f
p -> \u0070
q -> \u0071
r -> \u0072
s -> \u0073
t -> \u0074
u -> \u0075
v -> \u0076
w -> \u0077
x -> \u0078
y -> \u0079
z -> \u007a


In [49]:
puncs += r"\u" + f"{ord('।'):04x}"
print(puncs)

;:,.!?—…"«»“” \u0964


In [28]:
int("9e0", 16)

2528

In [44]:
i = [x for x in range(int("981",16), int(""))] + [y for y in range(5, 10)]
i

[1, 2, 5, 6, 7, 8, 9]

In [50]:
for i in range(int("981",16), int("9e0",16)):
    print(f"\\u{i:04x} -> {chr(i)}")
    chars += f"\\u{i:04x}"

\u0981 -> ঁ
\u0982 -> ং
\u0983 -> ঃ
\u0984 -> ঄
\u0985 -> অ
\u0986 -> আ
\u0987 -> ই
\u0988 -> ঈ
\u0989 -> উ
\u098a -> ঊ
\u098b -> ঋ
\u098c -> ঌ
\u098d -> ঍
\u098e -> ঎
\u098f -> এ
\u0990 -> ঐ
\u0991 -> ঑
\u0992 -> ঒
\u0993 -> ও
\u0994 -> ঔ
\u0995 -> ক
\u0996 -> খ
\u0997 -> গ
\u0998 -> ঘ
\u0999 -> ঙ
\u099a -> চ
\u099b -> ছ
\u099c -> জ
\u099d -> ঝ
\u099e -> ঞ
\u099f -> ট
\u09a0 -> ঠ
\u09a1 -> ড
\u09a2 -> ঢ
\u09a3 -> ণ
\u09a4 -> ত
\u09a5 -> থ
\u09a6 -> দ
\u09a7 -> ধ
\u09a8 -> ন
\u09a9 -> ঩
\u09aa -> প
\u09ab -> ফ
\u09ac -> ব
\u09ad -> ভ
\u09ae -> ম
\u09af -> য
\u09b0 -> র
\u09b1 -> ঱
\u09b2 -> ল
\u09b3 -> ঳
\u09b4 -> ঴
\u09b5 -> ঵
\u09b6 -> শ
\u09b7 -> ষ
\u09b8 -> স
\u09b9 -> হ
\u09ba -> ঺
\u09bb -> ঻
\u09bc -> ়
\u09bd -> ঽ
\u09be -> া
\u09bf -> ি
\u09c0 -> ী
\u09c1 -> ু
\u09c2 -> ূ
\u09c3 -> ৃ
\u09c4 -> ৄ
\u09c5 -> ৅
\u09c6 -> ৆
\u09c7 -> ে
\u09c8 -> ৈ
\u09c9 -> ৉
\u09ca -> ৊
\u09cb -> ো
\u09cc -> ৌ
\u09cd -> ্
\u09ce -> ৎ
\u09cf -> ৏
\u09d0 -> ৐
\u09d1 -> ৑
\u09d2 -> ৒
\u09d3 -> ৓
\u09

In [51]:
print(chars)

@ABCDEFGHIJKLMNPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz\u0981\u0982\u0983\u0984\u0985\u0986\u0987\u0988\u0989\u098a\u098b\u098c\u098d\u098e\u098f\u0990\u0991\u0992\u0993\u0994\u0995\u0996\u0997\u0998\u0999\u099a\u099b\u099c\u099d\u099e\u099f\u09a0\u09a1\u09a2\u09a3\u09a4\u09a5\u09a6\u09a7\u09a8\u09a9\u09aa\u09ab\u09ac\u09ad\u09ae\u09af\u09b0\u09b1\u09b2\u09b3\u09b4\u09b5\u09b6\u09b7\u09b8\u09b9\u09ba\u09bb\u09bc\u09bd\u09be\u09bf\u09c0\u09c1\u09c2\u09c3\u09c4\u09c5\u09c6\u09c7\u09c8\u09c9\u09ca\u09cb\u09cc\u09cd\u09ce\u09cf\u09d0\u09d1\u09d2\u09d3\u09d4\u09d5\u09d6\u09d7\u09d8\u09d9\u09da\u09db\u09dc\u09dd\u09de\u09df


In [54]:
f"{8217:04x}"

'2019'

In [52]:
ord('’')

8217

# Audio Cleaning

Utility code for cleaning a set of audio files (using pydub).

Operations to perform:
* Normalizing Amplitude
* Converting from Stereo to Mono
* Adjusting Sampling Rate

In [8]:
src_dir = "/home/arif/Dropbox/PhD/Resource/My Thesis Report/sust-tts-cmos/wavs"
dest_dir = "/home/arif/Dropbox/PhD/Resource/My Thesis Report/sust-tts-cmos/wavs-updated"

In [9]:
files = os.listdir(src_dir)
len(files)

12

In [10]:
total = 0
for file in files:
    wavfile = os.path.join(src_dir, file)
    
    audio = AudioSegment.from_file(wavfile)
    
#     print(audio.dBFS)
    audio = audio.apply_gain(-18 - audio.dBFS)
    print(audio.dBFS)
    
    audio = audio.set_channels(1)
    #print(audio.channels)
    
    audio = audio.set_frame_rate(22050)
    #print(audio.frame_rate)
    
    audio.export(os.path.join(dest_dir, file), format="wav")
    total += 1
    
print(f'{total} files processed.')

-18.000519641475478
-18.000519641475478
-17.9984142264517
-18.000519641475478
-18.000519641475478
-17.9984142264517
-18.000519641475478
-18.000519641475478
-18.000519641475478
-17.9984142264517
-18.000519641475478
-18.000519641475478
12 files processed.


# Creating adrita16 & kanak27
description ...

In [17]:
wav_dir = "/media/arif/Resource/Etc/data/kanak22/wavs"
files = os.listdir(wav_dir)
files = sorted(files)
len(files)

15383

In [19]:
metafile = "/media/arif/Resource/Etc/data/kanak22/metadata.csv"
texts = []
with open(metafile) as f:
    for line in f:
        #print(line)
        idx, _, text = line.split('|')
        texts.append((idx.strip(), text.strip()))

texts = sorted(texts)
len(texts)

15383

In [12]:
texts[0]

('10000', 'স্কুলটির দিকে দৃষ্টি দিন।')

In [20]:
i = 0
total_duration = 0
max_duration = 0
min_duration = 100000000
max_chars = 0
min_chars = 100000000
dest_dir = '/media/arif/Resource/Etc/data/kanak22/'
with open(os.path.join(dest_dir, 'metadata.csv'), 'r') as outfile:
    for file, text_item in zip(files, texts):
        idx, text = text_item

        #print(f"filename: {file[:-4]} \t id: {idx}")
        assert(file[:-4] == idx)

        audio = AudioSegment.from_wav(os.path.join(wav_dir, file))

        duration = audio.duration_seconds
        if duration < 1 or duration > 8: continue

        total_duration += duration
        max_duration = max(max_duration, duration)
        min_duration = min(min_duration, duration)
        max_chars = max(max_chars, len(text))
        min_chars = min(min_chars, len(text))

#         shutil.copy(os.path.join(wav_dir, file), os.path.join(dest_dir, 'wavs', file))
#         outfile.write(idx + '| |' + text + '\n')

        i += 1
#         if i == 2: break
        
avg_duration = total_duration / i
total_duration /= 60 # sec. to min.
duration_hours = total_duration // 60
duration_mins = total_duration - (duration_hours * 60)

print(f'Total samples: {i}')
print(f"Total duration: {int(duration_hours)} hours {duration_mins:.2f} minutes")
print(f'Maximum duration: {max_duration:.2f} seconds')
print(f'Minimum duration: {min_duration:.2f} seconds')
print(f'Average duration: {avg_duration:.2f} seconds') 
print(f'Maximum text length: {max_chars}')
print(f'Minimum text length: {min_chars}')

Total samples: 13071
Total duration: 16 hours 37.81 minutes
Maximum duration: 8.00 seconds
Minimum duration: 1.00 seconds
Average duration: 4.58 seconds
Maximum text length: 115
Minimum text length: 9


In [8]:
i

11959

## Removing ZWNJ and ZWJ from the text corpora

In [2]:
# -*- coding: utf-8 -*-
"""Utility for context-aware removal of ZWNJ and ZWJ in Bangla text."""

from __future__ import unicode_literals

import io
import re
import unittest


STANDARDIZE_ZW = re.compile(r'(?<=\u09b0)[\u200c\u200d]+(?=\u09cd\u09af)')

DELETE_ZW = re.compile(r'(?<!\u09b0)[\u200c\u200d](?!\u09cd\u09af)')


def RemoveOptionalZW(text):
    """Removes all optional occurrences of ZWNJ or ZWJ from Bangla text.

    The non-printing characters U+200C (ZWNJ) and U+200D (ZWJ) are used in Bangla
    to optionally control the appearance of ligatures, except in one special
    situation: after RA and before VIRAMA+YA, the presence or absence of ZWJ
    (formerly ZWNJ) changes the visual appearance of the involved consonants in a
    meaningful way. This occurrences of ZWJ must be preserved, while all other
    occurrences are advisory and can be removed for most purposes.

    After RA and before VIRAMA+YA, this function changes ZWNJ to ZWJ and preserves
    ZWJ; and removes ZWNJ and ZWJ everywhere else.

    Args:
    text: The text from which the zero-width format controls are to be removed.

    Returns:
    The text with all non-obligatory occurrences of ZWNJ and ZWJ removed.

    """
    text = STANDARDIZE_ZW.sub('\u200D', text)
    text = DELETE_ZW.sub('', text)
    return text


class TestBanglaZeroWidth(unittest.TestCase):

    def test_RemoveOptionalZW(self):
        rally = '\u09b0\u200d\u09cd\u09af\u09be\u09b2\u09c0'
        self.assertEqual(rally, 'র‍্যালী')
        self.assertEqual(RemoveOptionalZW(rally), rally)
        for i in range(len(rally) + 1):
            for zw in ('\u200c', '\u200d', '\u200c\u200d', '\u200d\u200c'):
                text = rally[:i] + zw + rally[i:]
                self.assertEqual(RemoveOptionalZW(text), rally)


if __name__ == '__main__':
    stdin = io.open(0, mode='rt', encoding='utf-8', closefd=False)
    stdout = io.open(1, mode='wt', encoding='utf-8', closefd=False)
    for line in stdin:
        stdout.write(RemoveOptionalZW(line))

In [14]:
csv_file = "/media/arif/Resource/Etc/data/kanak30/metadata.csv"
updated_csv = "/media/arif/Resource/Etc/data/kanak30/update.csv"

i = 0
with open(csv_file) as f:
    with open(updated_csv, 'w') as out:
        for line in f: 
            line = RemoveOptionalZW(line)

            if '\u200c' in line or '\u200d' in line:
                print(line.strip())

01220| |ফজলে হাসান আবেদ সামাজিক উন্নয়নে অসামান্য ভূমিকার জন্য র‍্যামন ম্যাগসেসে পুরস্কার পান।
06780| |পুলিশ এই র‍্যালি অবরোধ করেছিল।
10334| |হান্নান র‍্যাবের কাছে অনেক তথ্য জানালেও সে স্বীকারোক্তিমূলক জবানবন্দি দিতে রাজি হয়নি বলে জানা গেছে।
12163| |হয়তো এটা পাগল - হয়তো এটা র‍্যাবিড।
12703| |আমাদের একটা পথ আছে, সেটার গন্তব্য র‍্যাডিসন হোটেল বা মার্কিন রাষ্ট্রদূতের বাসভবন নয়।
13492| |এদিকে গ্রেপ্তার করা মুফতি হান্নানকে র‍্যাব এখনো জিজ্ঞাসাবাদ করছে।
14052| |তাঁর স্ত্রী আরজিনা বেগমকে র‍্যাব আটক করেছে।
14626| |এ ঘটনায় জড়িত থাকার অভিযোগে আমির উদ্দিনকে আটক করে র‍্যাব।
16023| |মজাদার দেশি খাবার, শাড়ি গয়নার স্টল, নাচ গান, র‍্যাফেল ড্র আর আড্ডায় ভরপুর অনুষ্ঠানটি দেখতে দেখতেই শেষ হয়ে যায়।
16385| |সেখানে তাঁরা জুটি হয়ে র‍্যাম্পে হাঁটেন।
16421| |বাকি ব্যক্তিদের মধ্যে সাতজন পুলিশের সঙ্গে ও একজন র‍্যাবের সঙ্গে বন্দুকযুদ্ধে নিহত হন।
16447| |এ ছাড়া ডুয়িং বিজনেস প্রতিবেদনেও বাংলাদেশের র‍্যাঙ্কিংয়ের উন্নতি হবে।
16662| |কুমিল্লা, সিরাজগঞ্জ ও রাজবাড়ীতে পুলিশ ও র‍্যাবের সঙ্গে বন্দুকযুদ্ধে চার ব্যক্তি নিহ

## SUST TTS Evaluation Test Set Texts

In [2]:
textfile = "/home/arif/Dropbox/PhD/Resource/My Thesis Report/test-set.csv"

In [11]:
words = []
total = 0
with open(textfile) as f:
    for line in f:
        idx, txt = line.split('\t')
        
        txt = txt.replace(',', ' ').replace(';', ' ').replace('।', ' ').replace('?', ' ').replace('!', ' ').replace('-', ' ').strip()
        #print(txt)
        
        tokens = txt.split()
        words.extend(tokens)
        
        total += 1
        #if total == 15: break


with open("test-words.csv", "w") as out:
    for word in words:
        print(word)
        out.write(word + '\n')
print(f'Total: {len(words)} words.')

আমাদের
মাতৃভূমির
যে
মাটিতে
আমরা
দাঁড়িয়ে
আছি
ওপরে
তাকালে
যে
আকাশ
আমরা
দেখতে
পাই
কিংবা
নিঃশ্বাসে
যে
বাতাস
আমরা
বুকের
ভেতর
টেনে
নেই
তার
সবকিছুর
জন্যেই
আমরা
আমাদের
মুক্তিযোদ্ধাদের
কাছে
ঋণী
এটি
একটি
বাংলা
বাক্যের
উদাহরণ
মানুষের
যতগুলো
অনুভূতি
আছে
তার
মাঝে
সবচেয়ে
সুন্দর
অনুভূতি
হচ্ছে
ভালোবাসা
বাংলাদেশের
প্রথম
ও
পূর্ণাঙ্গ
সার্চ
ইঞ্জিন
হিসেবে
পিপীলিকা
আত্মপ্রকাশ
করে
দীর্ঘ
এক
যুগ
পর
বহুল
প্রতীক্ষিত
এ
সমাবর্তন
ঘিরে
ক্যাম্পাসে
চলছে
শেষ
মুহুর্তের
প্রস্তুতি
ক্লাস
পরীক্ষার
ফাঁকে
টং
দোকানগুলোই
শিক্ষার্থীদের
আড্ডার
প্রাণকেন্দ্র
রাষ্ট্রের
বিভিন্ন
কার্যাবলী
পরিচালনার
জন্য
বিভিন্ন
মন্ত্রণালয়
রয়েছে
তাঁর
কি
করার
কিছুই
ছিল
না
এই
অত্যাচার
এর
বিরুদ্ধে
আপনারা
কি
কিছুই
করতে
পারছেন
না
চালাক
আর
বুদ্ধিমান
এক
নয়
এটা
জানো
তো
এদিকে
গ্রেফতার
করা
মুফতি
হান্নানকে
র‍্যাব
এখনো
জিজ্ঞাসাবাদ
করছে
শহরে
একটি
বিশ্ববিদ্যালয়
একাধিক
চিকিৎসা
ইন্সটিটিউট
এবং
সামুদ্রিক
স্থাপত্যের
একটি
স্কুল
আছে
গ্রিসের
চিকিৎসক
বেশ
কয়েকদিন
ধরে
সব
ধরনের
পরীক্ষা
করলেন
বাদশাহকে
ঈশ্বর
প্রত্যেক
মানুষকে
দিয়েছেন
দেহ
মন
ও
আত্মা
বিদ্যালয়ের
আনুষ্ঠানিক
শিক্ষাল

## Batch PESQ

In [2]:
wavdir = "/home/arif/Dropbox/PhD/Resource/My Thesis Report/pesq"

In [7]:
wavfiles = sorted(os.listdir(wavdir))
len(wavfiles)

64

In [15]:
for file in wavfiles:
        wavfile = os.path.join(wavdir, file)
    
        audio = AudioSegment.from_file(wavfile)

        audio = audio.set_channels(1)
        #print(audio.channels)

        audio = audio.set_frame_rate(16000)
        #print(file, audio.frame_rate, audio.channels)
        
        if "_" not in file:
            ref = audio.get_array_of_samples()
            ref = np.array(ref)
            #print(file, ref.frame_rate)
            continue
            
        target = audio.get_array_of_samples()
        target = np.array(target)
        
        pesq_n = pesq(16000, ref, target, 'nb')
        lqo = 1 + ((5 - 1) / (4.5 - -0.5)) * (pesq_n - -0.5)
        
        print(f'{file}: \t {lqo:.3f}')
    

06535_kn30_align_gl.wav: 	 2.298
06535_kn30_align_hifi.wav: 	 2.294
06535_kn30_align_mbmel.wav: 	 2.298
06535_kn30_align_univ.wav: 	 2.286
06535_kn30_fp_wgrad.wav: 	 2.246
06535_kn30_glow_hifi.wav: 	 2.239
06535_kn30_t2ddc_hifi.wav: 	 2.257
06535_kn30_vits.wav: 	 2.232
08438_kn30_align_gl.wav: 	 2.336
08438_kn30_align_hifi.wav: 	 2.226
08438_kn30_align_mbmel.wav: 	 2.308
08438_kn30_align_univ.wav: 	 2.226
08438_kn30_fp_wgrad.wav: 	 2.260
08438_kn30_glow_hifi.wav: 	 2.242
08438_kn30_t2ddc_hifi.wav: 	 2.297
08438_kn30_vits.wav: 	 2.228
10104_al12_align_gl.wav: 	 2.506
10104_al12_align_hifi.wav: 	 2.669
10104_al12_align_mbmel.wav: 	 2.588
10104_al12_align_univ.wav: 	 2.827
10104_al12_align_wgrad.wav: 	 2.503
10104_al12_align_wrnn.wav: 	 2.532
10104_al12_fp_hifi.wav: 	 2.447
10104_al12_glow_hifi.wav: 	 2.358
10104_al12_t2dca_hifi.wav: 	 2.400
10104_al12_t2ddc_hifi.wav: 	 2.247
10104_al12_vits.wav: 	 2.327
10129_al12_align_gl.wav: 	 2.620
10129_al12_align_hifi.wav: 	 2.544
10129_al12_align_